## Inventory Health Analysis
- Objective: To assess inventory pressure and identify SKUs where pricing decisions are constrained or enabled by stock conditions.

In [1]:
# Load & Inspect Inventory Data
import pandas as pd
import numpy as np

inventory_df = pd.read_csv("dataset/Inventory_Health.csv")

inventory_df.head()

,SKU,condition,total-inventory,available,inbound-shipped,inbound-received,reserved-quantity,unfulfillable-quantity,inv-age-0-to-30-days,inv-age-31-to-60-days,...,units-shipped-t90,sell-through,item-volume,volume-unit-measurement,storage-type,storage-volume,marketplace,days-of-supply,weeks-of-cover-t30,weeks-of-cover-t90
0,MN-01,New,382.0,264,40,9,68,1,12,261,...,574,1.33,0.546781,cubic feet,Standard,144.350184,US,47,6,7
1,MN-02,New,306.0,206,80,2,18,0,127,91,...,417,1.56,0.373971,cubic feet,Standard,77.038026,US,85,7,8
2,MN-03,New,139.0,95,40,0,3,1,42,8,...,179,1.16,0.425846,cubic feet,Standard,40.455370,US,83,8,9
3,MN-04,New,171.0,70,80,8,13,0,3,84,...,187,1.65,0.375575,cubic feet,Standard,26.290250,US,99,8,10
4,MN-05,New,283.0,0,0,49,234,0,53,141,...,557,1.47,0.270417,cubic feet,Standard,0.000000,US,145,0,1


In [2]:
inventory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SKU                      50 non-null     object 
 1   condition                50 non-null     object 
 2   total-inventory          48 non-null     float64
 3   available                50 non-null     int64  
 4   inbound-shipped          50 non-null     int64  
 5   inbound-received         50 non-null     int64  
 6   reserved-quantity        50 non-null     int64  
 7   unfulfillable-quantity   50 non-null     int64  
 8   inv-age-0-to-30-days     50 non-null     int64  
 9   inv-age-31-to-60-days    50 non-null     int64  
 10  inv-age-61-to-90-days    50 non-null     int64  
 11  inv-age-181-to-330-days  50 non-null     int64  
 12  inv-age-331-to-365-days  50 non-null     int64  
 13  inv-age-365-plus-days    50 non-null     int64  
 14  currency                 50 

In [3]:
inventory_df.isna().sum()

SKU                        0
condition                  0
total-inventory            2
available                  0
inbound-shipped            0
inbound-received           0
reserved-quantity          0
unfulfillable-quantity     0
inv-age-0-to-30-days       0
inv-age-31-to-60-days      0
inv-age-61-to-90-days      0
inv-age-181-to-330-days    0
inv-age-331-to-365-days    0
inv-age-365-plus-days      0
currency                   0
units-shipped-t7           0
units-shipped-t30          0
units-shipped-t60          0
units-shipped-t90          0
sell-through               0
item-volume                0
volume-unit-measurement    0
storage-type               0
storage-volume             0
marketplace                0
days-of-supply             0
weeks-of-cover-t30         0
weeks-of-cover-t90         0
dtype: int64

In [4]:
# Total Inventory Exposure
inventory_df["Total_Inventory"] = (
    inventory_df["available"]
    + inventory_df["inbound-shipped"]
    + inventory_df["inbound-received"]
)

In [5]:
inventory_df["Total_Inventory"].head()

0    313
1    288
2    135
3    158
4     49
Name: Total_Inventory, dtype: int64

In [6]:
# Net Sellable Inventory
inventory_df["Net_Sellable_Inventory"] = (
    inventory_df["available"] - inventory_df["reserved-quantity"]
)

In [7]:
inventory_df["Net_Sellable_Inventory"].head()

0    196
1    188
2     92
3     57
4   -234
Name: Net_Sellable_Inventory, dtype: int64

In [8]:
# Days of Supply Buckets
inventory_df["Inventory_Status"] = pd.cut(
    inventory_df["days-of-supply"],
    bins=[-1, 15, 45, 90, 1000],
    labels=["Understocked", "Healthy", "Overstocked", "Excess"]
)

In [9]:
inventory_df["Inventory_Status"].value_counts()

Inventory_Status
Overstocked     32
Excess          12
Understocked     3
Healthy          3
Name: count, dtype: int64

In [10]:

inventory_df["Old_Inventory_Pct"] = (
    (inventory_df["inv-age-181-to-330-days"] +
     inventory_df["inv-age-331-to-365-days"] +
     inventory_df["inv-age-365-plus-days"]) /
    inventory_df["Total_Inventory"]
).replace([np.inf], 0).fillna(0) * 100

In [11]:
# Weeks of Cover Stability
inventory_df["Coverage_Gap"] = (
    inventory_df["weeks-of-cover-t90"] - inventory_df["weeks-of-cover-t30"]
)

In [12]:
inventory_df["Coverage_Gap"].head()

0    1
1    1
2    1
3    2
4    1
Name: Coverage_Gap, dtype: int64

In [13]:
# Sell-Through Efficiency (Demand vs Inventory)
inventory_df["Sell_Through_Level"] = pd.qcut(
    inventory_df["sell-through"],
    q=3,
    labels=["Low", "Medium", "High"]
)

inventory_df["Sell_Through_Level"].value_counts()


Sell_Through_Level
Low       17
High      17
Medium    16
Name: count, dtype: int64

In [14]:
# SKU-level inventory pricing recommendations

inventory_df["Inventory_Pricing_Signal"] = "Hold"

inventory_df.loc[
    (inventory_df["Inventory_Status"].isin(["Overstocked", "Excess"])) &
    (inventory_df["Sell_Through_Level"] == "Low"),
    "Inventory_Pricing_Signal"
] = "Reduce Price"

inventory_df.loc[
    (inventory_df["Inventory_Status"] == "Understocked") &
    (inventory_df["Sell_Through_Level"] == "High"),
    "Inventory_Pricing_Signal"
] = "Increase Price"


If inventory is high and sales are slow, the model recommends a price reduction to clear stock. If inventory is low and sales are strong, it recommends a price increase to improve margins and control stock-outs. Otherwise, prices are held steady

In [15]:
# Risk Flags (Operational Reality)
inventory_df["Inventory_Risk_Flag"] = (
    inventory_df["unfulfillable-quantity"] >
    inventory_df["unfulfillable-quantity"].median()
)

In [16]:
inventory_df["Inventory_Risk_Flag"].value_counts()

Inventory_Risk_Flag
False    34
True     16
Name: count, dtype: int64

In [17]:
# Final Inventory Signal Table 
inventory_signals = inventory_df[[
    "SKU",
    "Total_Inventory",
    "Net_Sellable_Inventory",
    "days-of-supply",
    "Inventory_Status",
    "Sell_Through_Level",
    "Inventory_Pricing_Signal",
    "Inventory_Risk_Flag"
]]

inventory_signals.head()

,SKU,Total_Inventory,Net_Sellable_Inventory,days-of-supply,Inventory_Status,Sell_Through_Level,Inventory_Pricing_Signal,Inventory_Risk_Flag
0,MN-01,313,196,47,Overstocked,Low,Reduce Price,False
1,MN-02,288,188,85,Overstocked,Medium,Hold,False
2,MN-03,135,92,83,Overstocked,Low,Reduce Price,False
3,MN-04,158,57,99,Excess,Medium,Hold,False
4,MN-05,49,-234,145,Excess,Medium,Hold,False


In [18]:
inventory_signals.to_csv(
    "final_outputs/inventory_signals.csv",
    index=False
)

## Inventory Health Analysis – Key Insights
- Inventory pressure is not uniform across products
- Sell-through performance varies meaningfully across SKUs
- Overstocked SKUs with low sell-through are pricing risks
- Understocked SKUs with high sell-through show pricing power
- Inventory condition directly constrains pricing flexibility
- Inventory signals help distinguish “price problems” from “stock problems”